# imports

In [1]:
from transformers import AutoProcessor
import requests
import torch
from llava.conversation import conv_templates, SeparatorStyle
from transformers import CLIPImageProcessor
from llava.model import *
from llava.mm_utils import KeywordsStoppingCriteria
from llava.model.multimodal_encoder.builder import build_vision_tower
from io import BytesIO
from PIL import Image
from llava.mm_utils import tokenizer_image_token

# parameters

In [2]:
device = "cuda:0"
model_name = "tabtoyou/KoLLaVA-v1.5-Synatra-7b"

DEFAULT_IMAGE_TOKEN = "<image>"
DEFAULT_IMAGE_PATCH_TOKEN = "<im_patch>"
DEFAULT_IM_START_TOKEN = "<im_start>"
DEFAULT_IM_END_TOKEN = "<im_end>"
IMAGE_TOKEN_INDEX = -200

# utils

In [3]:
def load_image(image_file):
    if image_file.startswith('http') or image_file.startswith('https'):
        response = requests.get(image_file)
        image = Image.open(BytesIO(response.content)).convert('RGB')
    else:
        image = Image.open(image_file).convert('RGB')
    return image

# load

In [4]:
tokenizer = AutoProcessor.from_pretrained("tabtoyou/KoLLaVA-v1.5-Synatra-7b")
model = LlavaLlamaForCausalLM.from_pretrained(model_name, low_cpu_mem_usage=True, torch_dtype=torch.float16, use_cache=True).to(device)
image_processor = CLIPImageProcessor.from_pretrained(model.config.mm_vision_tower, torch_dtype=torch.float16)

You are using the legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This means that tokens that come after special tokens will not be properly handled. We recommend you to read the related pull request available at https://github.com/huggingface/transformers/pull/24565


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some weights of the model checkpoint at tabtoyou/KoLLaVA-v1.5-Synatra-7b were not used when initializing LlavaLlamaForCausalLM: ['model.vision_tower.vision_tower.vision_model.encoder.layers.21.self_attn.q_proj.bias', 'model.vision_tower.vision_tower.vision_model.encoder.layers.21.self_attn.v_proj.bias', 'model.vision_tower.vision_tower.vision_model.encoder.layers.12.layer_norm2.bias', 'model.vision_tower.vision_tower.vision_model.encoder.layers.20.self_attn.out_proj.weight', 'model.vision_tower.vision_tower.vision_model.encoder.layers.13.self_attn.q_proj.weight', 'model.vision_tower.vision_tower.vision_model.encoder.layers.1.mlp.fc2.bias', 'model.vision_tower.vision_tower.vision_model.encoder.layers.1.layer_norm1.bias', 'model.vision_tower.vision_tower.vision_model.encoder.layers.1.self_attn.v_proj.weight', 'model.vision_tower.vision_tower.vision_model.encoder.layers.12.mlp.fc1.bias', 'model.vision_tower.vision_tower.vision_model.encoder.layers.1.layer_norm2.bias', 'model.vision_tower.

# set model & tokenizer

In [7]:
# tokenizer
mm_use_im_start_end = getattr(model.config, "mm_use_im_start_end", False)
tokenizer.add_tokens([DEFAULT_IMAGE_PATCH_TOKEN], special_tokens=True)
tokenizer.add_tokens([DEFAULT_IM_START_TOKEN, DEFAULT_IM_END_TOKEN], special_tokens=True)

0

In [6]:
# vision tower
model.get_model().vision_tower = build_vision_tower(model.config, delay_load=False).to(device) # delayed load가 default로 되어 있어 외부에서 직접 정의
vision_tower = model.get_model().vision_tower
vision_config = vision_tower.config
vision_config.im_patch_token = tokenizer.convert_tokens_to_ids([DEFAULT_IMAGE_PATCH_TOKEN])[0]
vision_config.use_im_start_end = mm_use_im_start_end
vision_config.im_start_token, vision_config.im_end_token = tokenizer.convert_tokens_to_ids([DEFAULT_IM_START_TOKEN, DEFAULT_IM_END_TOKEN])
image_token_len = (vision_config.image_size // vision_config.patch_size) ** 2

# inference

In [8]:
def inference(qs, image_file) :
    image = load_image(image_file)
    qs = DEFAULT_IMAGE_TOKEN + '\n' + qs
    conv_mode = "mistral"
    conv = conv_templates[conv_mode].copy()
    conv.append_message(conv.roles[0], qs)
    conv.append_message(conv.roles[1], None)
    prompt = conv.get_prompt()
    print(prompt)
    image_tensor = image_processor.preprocess(image, return_tensors='pt')['pixel_values'][0]
    input_ids = tokenizer_image_token(prompt, tokenizer, IMAGE_TOKEN_INDEX, return_tensors='pt').unsqueeze(0)

    stop_str = conv.sep if conv.sep_style != SeparatorStyle.TWO else conv.sep2
    keywords = [stop_str, "###"]
    stopping_criteria = KeywordsStoppingCriteria(keywords, tokenizer, input_ids)

    with torch.inference_mode():
        output_ids = model.generate(
            input_ids.to(device),
            images=image_tensor.unsqueeze(0).half().to(device),
            do_sample=True,
            temperature=0.2,
            max_new_tokens=1024,
            stopping_criteria=[stopping_criteria]).cpu()

    input_token_len = input_ids.shape[1]
    n_diff_input_output = (input_ids != output_ids[:, :input_token_len]).sum().item()

    if n_diff_input_output > 0:
        print(f'[Warning] {n_diff_input_output} output_ids are not the same as the input_ids')
    outputs = tokenizer.batch_decode(output_ids[:, input_token_len:-1], skip_special_tokens=True)[0]
    outputs = outputs.strip()
    if outputs.endswith(stop_str):
        outputs = outputs[:-len(stop_str)]
    outputs = outputs.strip()
    print(outputs)

In [9]:
image_file = "images/dog.png"
query = "사진 속 동물은 무엇인가요?"

inference(qs=query, image_file=image_file)

당신은 유용한 언어 및 시각 도우미입니다. 사용자가 제공하는 시각적 콘텐츠를 이해할 수 있으며, 자연어를 사용하여 사용자에게 다양한 작업을 지원합니다. <|im_start|>user
: <image>
사진 속 동물은 무엇인가요? <|im_start|>assistant
:


/opt/conda/envs/updated/lib/python3.10/site-packages/transformers/generation/utils.py:1270: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(


사진 속 동물은 흰 개입니다.


In [10]:
image_file = "images/food.png"
query = "사진 속엔 어떤 음식들이 있나요?"

inference(qs=query, image_file=image_file)

당신은 유용한 언어 및 시각 도우미입니다. 사용자가 제공하는 시각적 콘텐츠를 이해할 수 있으며, 자연어를 사용하여 사용자에게 다양한 작업을 지원합니다. <|im_start|>user
: <image>
사진 속엔 어떤 음식들이 있나요? <|im_start|>assistant
:
이미지에는 냉장고 안에 있는 다양한 식품이 표시되어 있습니다. 여기에는 우유, 딸기, 블루베리, 당근, 요구르트, 빵, 딸기 등이 포함됩니다. 이러한 품목은 냉장고 내부의 다양한 선반에 배치되어 있습니다.
